In [1]:
import pandas as pd
import csv
import sys
import requests
from lxml import html as htmlRenderer
csv.field_size_limit(sys.maxsize)

131072

In [15]:
# csv_file_in = "/home/cflores/cflores_workspace/comments-retriever/results/abc/abc-01012019-15092019-contents.csv"
# csv_file_out = "/home/cflores/cflores_workspace/comments-retriever/results/abc/abc-01012019-15092019-contents-tmp.csv"

In [50]:
csv_file_out = "/home/cflores/cflores_workspace/comments-retriever/results/abc/abc-01012019-15092019-contents-tmp-2.csv"
csv_file_in = "/home/cflores/cflores_workspace/comments-retriever/results/abc/abc-01012019-15092019-contents-tmp.csv"

In [51]:
def is_valid_csv(csv_file):
    invalid_rows = []
    no_content = []
    no_title = []
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        counter = 0
        for r in reader:
            if not r[0]:
                print(" row -{}- row has not url.".format(counter))
                invalid_rows.append(counter)
            if not r[1]: 
                print(" row -{}- row has not content.".format(counter))
                invalid_rows.append(counter)
                no_content.append(r[0])
            if not r[2] or r[2] == r[0]:
                print(" row -{}- row has not title.".format(counter))
                invalid_rows.append(counter)
                no_title.append(r[0])
            counter = counter + 1
        
    return len(list(dict.fromkeys(invalid_rows))) == 0, invalid_rows, no_content, no_title

In [22]:
valid, invalid_rows, no_content, no_title = is_valid_csv(csv_file_in)

 row -56- row has not content.
 row -57- row has not content.
 row -60- row has not content.
 row -84- row has not content.
 row -122- row has not content.
 row -122- row has not title.
 row -142- row has not content.
 row -144- row has not content.
 row -153- row has not content.
 row -154- row has not content.
 row -155- row has not content.
 row -158- row has not content.
 row -194- row has not content.
 row -195- row has not content.
 row -207- row has not content.
 row -211- row has not content.
 row -218- row has not content.
 row -222- row has not content.
 row -258- row has not content.
 row -276- row has not content.
 row -277- row has not content.
 row -288- row has not content.
 row -345- row has not content.
 row -357- row has not content.
 row -363- row has not content.
 row -364- row has not content.
 row -388- row has not content.
 row -393- row has not content.
 row -407- row has not content.
 row -411- row has not content.
 row -428- row has not content.
 row -438- row

In [23]:
len(no_content)

2889

In [24]:
len(no_title)

8

In [25]:
no_title

['https://www.abc.es/espana/abci-no-dara-presidencia-pp-si-no-retiran-medidas-contra-violencia-genero-201901021412_noticia.html',
 'https://www.abc.es/espana/abci-dirigentes-psoe-a-acuden-escrache-contra-cambio-201901160340_noticia.html',
 'https://www.abc.es/sociedad/abci-brigadista-asturiano-mineros-mas-cien-cien-sacar-julen-pozo-201901250405_noticia.html',
 'https://www.abc.es/historia/abci-extrana-y-desconocida-charla-entre-portavoz-hitler-y-espia-frances-tren-espanol-201905040100_noticia.html',
 'https://www.abc.es/espana/abci-juez-caso-andalucia-maria-nunez-baja-motivos-personales-201906181619_noticia.html',
 'https://www.abc.es/espana/abci-juez-caso-andalucia-maria-nunez-baja-salud-201906181550_noticia.html',
 'https://www.abc.es/ciencia/abci-todo-no-sabian-viaje-y-podria-haberles-matado-201907200327_noticia.html',
 'https://www.abc.es/sociedad/abci-carne-mecha-comercializada-como-marca-blanca-tambien-infectada-listeriosis-201908241614_noticia.html']

In [14]:
"https://www.abc.es/espana/abci-dirigentes-psoe-a-acuden-escrache-contra-cambio-201901160340_noticia.html" in no_content

True

In [52]:
def extract_content(url=None):
    response = requests.get(url)
    renderedPage = htmlRenderer.fromstring(response.text)
    queries_xpath = ["//span[@class='cuerpo-texto ']//p",
                     "//span[@class='cuerpo-texto  unreg']/p",
                     "//span[@class='cuerpo-texto']/p",
                     "//div[@id='tab_cronica']/p", 
                     "//span[@class='cuerpo-texto top-nuevo']//p", 
                     "//span[@class='contenido-top']//p", 
                     "//div[@id='tab_directo']//p",
                     "//span[@class='cuerpo-texto directo']//p"]
    contentStr = ""
    for q in queries_xpath:
        commentsElList = renderedPage.xpath(q)
        if len(commentsElList) > 0:
            contentArr = []
            for p in commentsElList:
                contentArr.append(p.text_content())
            contentStr = "".join([parrafo for parrafo in contentArr])
            break
    if not contentStr:
        print("\t -> url has not content found {}".format(url))
    return contentStr

In [53]:
def get_title(url):
    queryXpath = "//article//h1"
    queries_xpath = ["//span[@class='titular']",
                     "//h1[@class='titular']", 
                     "//h1[@class='titular principal']"]
    response = requests.get(url)
    renderedPage = htmlRenderer.fromstring(response.text)
    el = renderedPage.xpath(queryXpath)
    title = url
    for q in queries_xpath:
        el = renderedPage.xpath(q)
        if len(el) > 0:
            title = el[0].text if el[0].text != None else el[0].text_content()
            break
    if title == url:
        print (" \t -> title not found for url {}".format(url))
    return title

    

In [54]:
keys = []
no_content = []
no_title = []
with open(csv_file_in, 'r') as f:
    with open(csv_file_out, 'w') as fout:
        writer = csv.writer(fout)
        reader = csv.reader(f)
        for r in reader:
            if not r[1]:
                # no content
                print(" => row with url {} has any content".format(r[0]))
                content = extract_content(r[0])
                if content:
                    print(" \t ===>>> content found ")
                r[1] = content
            else:
                r[1] = r[1].replace("\t", "").replace("\n", "").lstrip().rstrip()
            if not r[2] or r[2] == r[0]:
                # no title
                print(" => row with url {} has any title".format(r[0]))
                r[2] = get_title(r[0])
            
            writer.writerow(r)

 => row with url https://www.abc.es/espana/abci-no-dara-presidencia-pp-si-no-retiran-medidas-contra-violencia-genero-201901021412_noticia.html has any content
	 -> url has not content found https://www.abc.es/espana/abci-no-dara-presidencia-pp-si-no-retiran-medidas-contra-violencia-genero-201901021412_noticia.html
 => row with url https://www.abc.es/espana/abci-no-dara-presidencia-pp-si-no-retiran-medidas-contra-violencia-genero-201901021412_noticia.html has any title
 	 -> title not found for url https://www.abc.es/espana/abci-no-dara-presidencia-pp-si-no-retiran-medidas-contra-violencia-genero-201901021412_noticia.html
 => row with url https://www.abc.es/sociedad/abci-campana-90kmh-201901041600_video.html has any content
	 -> url has not content found https://www.abc.es/sociedad/abci-campana-90kmh-201901041600_video.html
 => row with url https://www.abc.es/opinion/mingote/abci-mingote-y-dietas-201901080321_noticia.html has any content
	 -> url has not content found https://www.abc.es

In [55]:
valid, invalid_rows, no_content, no_title = is_valid_csv(csv_file_out)

 row -122- row has not content.
 row -122- row has not title.
 row -357- row has not content.
 row -921- row has not content.
 row -989- row has not content.
 row -2134- row has not content.
 row -2134- row has not title.
 row -2305- row has not content.
 row -2311- row has not content.
 row -2357- row has not content.
 row -2588- row has not content.
 row -2602- row has not content.
 row -2607- row has not content.
 row -3028- row has not content.
 row -3082- row has not content.
 row -3506- row has not content.
 row -3506- row has not title.
 row -3585- row has not content.
 row -4125- row has not content.
 row -5618- row has not content.
 row -6082- row has not content.
 row -7458- row has not content.
 row -7737- row has not content.
 row -7936- row has not content.
 row -8393- row has not content.
 row -9386- row has not content.
 row -9923- row has not content.
 row -10161- row has not content.
 row -14175- row has not content.
 row -15494- row has not content.
 row -15691- row h

In [56]:
len(no_content)

63

In [57]:
no_content

['https://www.abc.es/espana/abci-no-dara-presidencia-pp-si-no-retiran-medidas-contra-violencia-genero-201901021412_noticia.html',
 'https://www.abc.es/sociedad/abci-campana-90kmh-201901041600_video.html',
 'https://www.abc.es/opinion/mingote/abci-mingote-y-dietas-201901080321_noticia.html',
 'https://www.abc.es/deportes/futbol/copa-del-rey-videos/abci-copa-resumen-y-goles-sporting-valencia-201901090000_video.html',
 'https://www.abc.es/espana/abci-dirigentes-psoe-a-acuden-escrache-contra-cambio-201901160340_noticia.html',
 'https://www.abc.es/deportes/futbol/copa-del-rey-videos/abci-copa-octavos-final-resumen-y-goles-valencia-sporting-201901152342_video.html',
 'https://www.abc.es/opinion/mingote/abci-mingote-visto-mingote-201901170405_noticia.html',
 'https://www.abc.es/historia/abci-fortalezas-imperio-espanal-america-solidas-defensas-frenaron-piratas-y-ingleses-201602190424_noticia.html',
 'https://www.abc.es/deportes/futbol/liga-primera-division-videos/abci-liga-santander-resumen-y-